### Prediction gap test for regression= true_labels= true_labels= true_labels

### Imports

In [1]:
import pandas as pd
import os
import xgboost as xgb
from pathlib import Path
import json
from src.decision_tree.tree import parse_xgboost_dump,  TreeEnsemble
import numpy as np

In [2]:
while "notebooks" in os.getcwd():
    os.chdir("../")
os.chdir("src")

### Wine data

In [3]:
model_path = Path("../models")
model_name = "winequality_red"

In [4]:
trees = parse_xgboost_dump(model_path / f"{model_name}_dumped.txt")
some_trees = TreeEnsemble(trees.trees, model_path / f"{model_name}_saved.json")
#some_trees = TreeEnsemble(trees.trees[0:10])
df = pd.read_csv("../data/wine_quality/test_winequality_red_scaled.csv")
df[df.select_dtypes(np.float64).columns] = df.select_dtypes(np.float64).astype(np.float32)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320 entries, 0 to 319
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed_acidity         320 non-null    float32
 1   volatile_acidity      320 non-null    float32
 2   citric_acid           320 non-null    float32
 3   residual_sugar        320 non-null    float32
 4   chlorides             320 non-null    float32
 5   free_sulfur_dioxide   320 non-null    float32
 6   total_sulfur_dioxide  320 non-null    float32
 7   density               320 non-null    float32
 8   pH                    320 non-null    float32
 9   sulphates             320 non-null    float32
 10  alcohol               320 non-null    float32
 11  quality               320 non-null    int64  
dtypes: float32(11), int64(1)
memory usage: 16.4 KB


### Sprawdzenie eval() 

In [6]:
gbdt_model = xgb.Booster()
gbdt_model.load_model(model_path / f"{model_name}_saved.json")

true_labels = df.iloc[:, -1]
adam_trees_eval = [some_trees.eval(df.iloc[i, :-1]) for i in range(len(df))]

x_dmatrix = xgb.DMatrix(df.iloc[:, :-1])
actual_xgb_model_eval = gbdt_model.predict(x_dmatrix)

from numpy import exp

pd.DataFrame(zip(true_labels, adam_trees_eval, actual_xgb_model_eval, adam_trees_eval - actual_xgb_model_eval),
             columns=["Quality", "Eval by Adam Trees",
                      "Actual eval by XGB model", 'diff'])

/home/mateusz/Documents/tmp/prediction-gap/venv/lib/python3.10/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/home/mateusz/Documents/tmp/prediction-gap/venv/lib/python3.10/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/home/mateusz/Documents/tmp/prediction-gap/venv/lib/python3.10/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/home/mateusz/Documents/tmp/prediction-gap/venv/lib/python3.10/site-packages/xgboost/data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a

,Quality,Eval by Adam Trees,Actual eval by XGB model,diff
0,5,5.075910,5.075910,0.000000e+00
1,5,5.015626,5.015625,9.536743e-07
2,6,4.422324,4.422326,-2.384186e-06
3,6,5.984006,5.984005,1.430511e-06
4,6,5.956180,5.956178,1.430511e-06
...,...,...,...,...
315,4,6.572219,6.572220,-4.768372e-07
316,5,5.049266,5.049266,0.000000e+00
317,6,5.927140,5.927141,-4.768372e-07
318,5,4.972984,4.972983,9.536743e-07


~~Z jakiegoś powodu powstają małe różnice, które wraz ze wzrostem liczby drzew zaczynają się zwiększać i dochodzą do dosyć wyskich wartości jak na przykład 0.1~~ - błąd zmniejszony, chociaż jakaś mała niedokładność numeryczna wysępuje
    

### Housing data

In [7]:
model_path = Path("../models")
model_name = "housing"

In [10]:
trees = parse_xgboost_dump(model_path / f"{model_name}_dumped.txt")
some_trees = TreeEnsemble(trees.trees, model_path / f"{model_name}_saved.json")
#some_trees = TreeEnsemble(trees.trees[0:10])
df = pd.read_csv("../data/housing_data/test_housing_scaled.csv")
df = df.dropna() #No missing values are in training set, so i assume we also drop those in test set?
df[df.select_dtypes(np.float64).columns] = df.select_dtypes(np.float64).astype(np.float32)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4091 entries, 0 to 4127
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           4091 non-null   float32
 1   latitude            4091 non-null   float32
 2   housing_median_age  4091 non-null   float32
 3   total_rooms         4091 non-null   float32
 4   total_bedrooms      4091 non-null   float32
 5   population          4091 non-null   float32
 6   households          4091 non-null   float32
 7   median_income       4091 non-null   float32
 8   median_house_value  4091 non-null   float32
dtypes: float32(9)
memory usage: 175.8 KB


In [11]:
bdt_model = xgb.Booster()
gbdt_model.load_model(model_path / f"{model_name}_saved.json")

true_labels = df.iloc[:, -1]
adam_trees_eval = [some_trees.eval(df.iloc[i, :-1]) for i in range(len(df))]

x_dmatrix = xgb.DMatrix(df.iloc[:, :-1])
actual_xgb_model_eval = gbdt_model.predict(x_dmatrix)

from numpy import exp

pd.DataFrame(zip(true_labels, adam_trees_eval, actual_xgb_model_eval, adam_trees_eval - actual_xgb_model_eval),
             columns=["Quality", "Eval by Adam Trees",
                      "Actual eval by XGB model", 'diff'])

/home/mateusz/Documents/tmp/prediction-gap/venv/lib/python3.10/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/home/mateusz/Documents/tmp/prediction-gap/venv/lib/python3.10/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/home/mateusz/Documents/tmp/prediction-gap/venv/lib/python3.10/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/home/mateusz/Documents/tmp/prediction-gap/venv/lib/python3.10/site-packages/xgboost/data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a

,Quality,Eval by Adam Trees,Actual eval by XGB model,diff
0,77700.0,88779.914062,88779.929688,-0.015625
1,314300.0,325716.562500,325716.656250,-0.093750
2,99100.0,103045.085938,103045.054688,0.031250
3,109400.0,139330.468750,139330.281250,0.187500
4,76400.0,111683.140625,111683.132812,0.007812
...,...,...,...,...
4086,161500.0,180265.750000,180265.640625,0.109375
4087,87200.0,71559.000000,71559.085938,-0.085938
4088,112900.0,104490.453125,104490.453125,0.000000
4089,185100.0,206854.906250,206854.859375,0.046875
